In [1]:
import pandas as pd
import json
import zipfile
import numpy as np
import progressbar
import os
import pprint

In [111]:
def extraction_lien3(data):
    liens = []
    for ent in data:


        #personne morale
        if ent['formality']['content'].get('natureCessation') is None:
            if  ent['formality']['content'].get('personneMorale'):
                if ent['formality']['content']['personneMorale'].get('composition'):
                    pouvoirs = ent['formality']['content']['personneMorale']['composition']['pouvoirs']
                    lien = []
                    for item in pouvoirs: 
                        if item.get('typeDePersonne'):
                            if item['typeDePersonne'] == "INDIVIDU":
                                filter_item = item['individu']['descriptionPersonne']
                            else:
                                if item.get('entreprise'):
                                    if item.get('adresseEntreprise'):
                                        filter_item = item['entreprise']|item['adresseEntreprise']
                                    else:
                                        filter_item = item['entreprise']
                                else:
                                    if item.get('adresseEntreprise'):
                                        filter_item = item['adresseEntreprise']
                                    else:
                                        filter_item = {}
                            lien.append(filter_item)
                        else:
                            lien.append({})
                        
                    lien = pd.DataFrame(lien)
                    lien.columns = ['personneMorale.' + col for col in lien.columns]
                    lien['siren'] = ent['siren']

                    
                    liens.append(lien)

            #personne physique
            if ent['formality']['content'].get('personnePhysique'):           
                if not ent['formality']['content']['personnePhysique'].get('detailCessationEntreprise'):
                    if ent['formality']['content']['personnePhysique'].get('identite'):
                        if ent['formality']['content']['personnePhysique']['identite'].get('entrepreneur'):
                            if ent['formality']['content']['personnePhysique']['identite']['entrepreneur'].get('descriptionPersonne'):
                                temp = ent['formality']['content']['personnePhysique']['identite']['entrepreneur']['descriptionPersonne']
                                try :
                                    lien = pd.DataFrame([temp])
                                except:
                                    lien = pd.DataFrame(temp,[0])
                                lien.columns = ['personnePhysique.' + col for col in lien.columns]
                    else:
                        lien = pd.DataFrame()
                    
                    lien['siren'] = ent['siren']

                    liens.append(lien)
        
       
    return(pd.concat(liens, join='outer'))

#### lancement unitaire

In [119]:
list_file = os.listdir('formalite')

In [120]:
list_file = list_file[5:10]

liste_liens = []
with progressbar.ProgressBar(max_value=len(list_file)) as bar:
    for i, file in enumerate(list_file):
        data = open('formalite/'+file, encoding='utf8')
        data = json.load(data)
        liens = extraction_lien3(data)
        liens = liens[[col for col in liens.columns if not "Present" in col]]
        liste_liens.append(liens) 
        bar.update(i + 1)

100% (5 of 5) |##########################| Elapsed Time: 0:00:18 Time:  0:00:180:03


In [100]:
reseau = pd.concat(liste_liens, join='outer').reset_index().drop(columns=['index'])

In [156]:
#reseau.to_parquet('liens_entreprise.parquet', index=False)

#### bouclage

In [121]:
list_file = os.listdir('formalite')

In [127]:
def traitement(list_file):
    liste_liens = []
    with progressbar.ProgressBar(max_value=len(list_file)) as bar:
        for i, file in enumerate(list_file):
            data = open('formalite/'+file, encoding='utf8')
            data = json.load(data)
            liens = extraction_lien3(data)
            liens = liens[[col for col in liens.columns if not "Present" in col]]
            liste_liens.append(liens) 
            bar.update(i + 1)
    return(pd.concat(liste_liens, join='outer').reset_index().drop(columns=['index']))

In [123]:
traitement1 = traitement(list_file[:1000])
traitement1.to_parquet('liens_entreprise1.parquet', index=False)

100% (1000 of 1000) |####################| Elapsed Time: 3:01:38 Time:  3:01:383716


In [128]:
traitement2 = traitement(list_file[1000:2000])
traitement2.to_parquet('liens_entreprise2.parquet', index=False)

100% (1000 of 1000) |####################| Elapsed Time: 4:35:49 Time:  4:35:492806


In [129]:
traitement3 = traitement(list_file[2000:])
traitement3.to_parquet('liens_entreprise3.parquet', index=False)

100% (604 of 604) |######################| Elapsed Time: 4:19:26 Time:  4:19:260916


In [4]:
temp = []
for i in range(1,4):
    temp.append(pd.read_parquet(f'liens_entreprise{str(i)}.parquet') )

In [5]:
lien = pd.concat(temp)

In [7]:
lien.to_parquet('lien_entreprise_20250403.parquet')

#### Analyse

In [2]:
lien = pd.read_parquet('lien_entreprise_20250403.parquet')

In [4]:
lien.columns

Index(['personnePhysique.dateDeNaissance', 'personnePhysique.nom',
       'personnePhysique.prenoms', 'personnePhysique.nomUsage',
       'personnePhysique.nationalite', 'personnePhysique.codeNationalite',
       'personnePhysique.situationMatrimoniale', 'personnePhysique.siren',
       'siren', 'personneMorale.dateDeNaissance', 'personneMorale.role',
       'personneMorale.nom', 'personneMorale.prenoms',
       'personneMorale.nationalite', 'personneMorale.codeNationalite',
       'personneMorale.situationMatrimoniale', 'personneMorale.nomUsage',
       'personnePhysique.genre', 'personneMorale.roleEntreprise',
       'personneMorale.denomination', 'personneMorale.formeJuridique',
       'personneMorale.pays', 'personneMorale.codePays',
       'personneMorale.codePostal', 'personneMorale.commune',
       'personneMorale.codeInseeCommune', 'personneMorale.voie',
       'personneMorale.siren', 'personneMorale.genre',
       'personneMorale.complementLocalisation', 'personneMorale.lieuRe

In [3]:
lien[lien.siren=='878990365']

,personnePhysique.dateDeNaissance,personnePhysique.nom,personnePhysique.prenoms,personnePhysique.nomUsage,personnePhysique.nationalite,personnePhysique.codeNationalite,personnePhysique.situationMatrimoniale,personnePhysique.siren,siren,personneMorale.dateDeNaissance,...,personneMorale.caracteristiques,personneMorale.forCodePostal,personneMorale.forCommune,personneMorale.forCodeCommune,personnePhysique.dateEffetRoleDeclarant,personneMorale.codeApe,personneMorale.communeAncienne,personneMorale.communeDeRattachement,personneMorale.numDetenteur,personneMorale.numExploitant
7831860,None,None,None,None,None,None,None,None,878990365,1971-02,...,None,None,None,None,None,None,None,None,NaN,NaN
7831861,None,None,None,None,None,None,None,None,878990365,1971-05,...,None,None,None,None,None,None,None,None,NaN,NaN
